In [ ]:
GEO Readiness & Governance

In [ ]:
----------------------------Canonicalization---------------------------
Canonicalization = selecting one “official” URL for a piece of content when the same content is accessible through multiple URLs.
    All show the same content, but to AI agents,
    crawlers, and search engines these can look like different pages unless you specify the canonical.

Canonicalization ensures machines know exactly which version of a page is the source of truth — boosting trust, 
authority, and retrieval quality.

1. Why It Exists :::

    Because the web is messy. Most websites unintentionally generate duplicate content through:
    
        1. Tracking parameters
        
        2. HTTP vs HTTPS
        
        3. WWW vs non-WWW
        
        4. Pagination
        
        5. Filtering
        
        6. CMS auto-routes
        
        7. Session IDs
        
        8. Internal links pointing to slightly different URLs
    
    This creates multiple versions of the same page. That breaks:
    
        1. Trust
        
        2. Ranking
        
        3. Crawl budgets
        
        4. AI agent confidence
        
        5. Data consistency
    
    Canonicalization solves this.


Why GEO Readiness Cares : 

    AI agents (browsers, scrapers, model retrievers, search LLMs, ranking systems) need to know:

    “Which is the true, authoritative, machine-trusted version of this content?”

    Canonicalization gives a clear trust signal:

    "This URL is the master source. Use this."

    This increases machine certainty


Purpose in Enhancing Machine- Trust :::::
    
    Canonicalization improves:
    
    1. Content Authority
    
    Agents know which version is official → increases trust and confidence.
    
    2. Deduplication
    
    Avoids agents thinking you have 10 copies of the same thing.
    
    3. Stronger Indexing / Retrieval
    
    Search engines and AI crawlers allocate more reliable embeddings and ranking signals to your canonical.
    
    4. Better Policy Mapping
    
    Your llm.txt, agents.json, and metadata only need to be aligned with one URL — the canonical.
    
    5. Reduced Fragmentation
    
    Models don’t embed multiple versions of the same page with minor differences.
    
    6. Crawl Budget Efficiency
    
    Bots spend more time on important pages and not duplicates.



What Outcome/Enhancement You Get After Implementing ::::

we can confidently expect:

        Outcome 1: Higher Trust in AI Agents
        
        Your site will be seen as clean, unambiguous, authoritative.
        
        Outcome 2: Better Content Ranking & Retrieval
        
        LLMs map your content into embeddings more accurately → better surfacing.
        
        Outcome 3: Stronger Domain Identity
        
        Reduces noise and consolidates signals to the same “main” URL.
        
        Outcome 4: More Reliable Machine Understanding
        
        Agents can infer:
        
        primary topic
        
        primary entity
        
        primary author
        
        version to monitor
        
        policies to apply
        
        updates to track
        
        Outcome 5: Better GEO Score
        
        Canonicalization is a core mechanical integrity factor.
        Bad canonicalization = weak trust & governance score.

In [ ]:
# canonicalization.py


In [4]:
# Pillar 1, Sub-pillar 3
# See Bridge.ipynb cell 3 for logic
# ...existing code...
import requests
import xml.etree.ElementTree as ET
import gzip
from io import BytesIO
from urllib.parse import urlparse, urljoin, urldefrag
from bs4 import BeautifulSoup
import time
import collections

# Helper to print colored and formatted text for better readability
def print_header(text):
    print("\n" + "="*60)
    print(f" {text}")
    print("="*60)

def print_subheader(text):
    print("\n" + "-"*60)
    print(f" {text}")
    print("-"*60)

def print_status(message, status):
    padded_message = f"{message:<45}"
    if status == "FAIL" or status == "CRITICAL":
        status_str = f"[\033[91m{status}\033[0m]" # Red
    elif status == "WARN":
        status_str = f"[\033[93m{status}\033[0m]" # Yellow
    elif status == "PASS" or status == "INFO":
        status_str = f"[\033[92m{status}\033[0m]" # Green
    else:
        status_str = f"[{status}]"

    print(f"{padded_message} {status_str}")

def print_recommendation(rec):
    print(f"  - {rec}")

class CanonicalizationAnalyzer:
    """
    Analyzes a website's canonicalization and source singularity based on ARI v10.0 Pillar 1, Sub-pillar 3.
    """
    def __init__(self, base_url, max_pages_to_check=25):
        self.base_url = self._format_base_url(base_url)
        self.max_pages_to_check = max_pages_to_check
        self.urls_to_check = collections.deque()
        self.checked_urls = set()
        self.report = {
            "pages_checked": 0,
            "pages_with_canonical": 0,
            "pages_with_absolute_canonical": 0,
            "pages_with_valid_canonical_target": 0,
            "issues": [],
            "recommendations": [],
            "score": 0,
            "status": "Not Assessed"
        }
        self.session = requests.Session()
        self.session.headers.update({
            'User-Agent': 'ARI-Canonical-Analyzer/1.0',
            'Accept-Language': 'en-US,en;q=0.9',
            'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',
        })

    def _format_base_url(self, url):
        parsed = urlparse(url)
        if not parsed.scheme:
            url = "https://" + url
            parsed = urlparse(url)
        return f"{parsed.scheme}://{parsed.netloc}"

    def _fetch_url(self, url, method='GET'):
        try:
            if method == 'GET':
                response = self.session.get(url, timeout=10, allow_redirects=True)
            elif method == 'HEAD':
                response = self.session.head(url, timeout=10, allow_redirects=True)
            response.raise_for_status()
            return response
        except requests.exceptions.RequestException as e:
            self.report["issues"].append(f"Network error for {url}: {e}")
            return None

    def get_urls_from_sitemap(self):
        """Tries to get a list of URLs from the sitemap."""
        print_status("Attempting to fetch URLs from sitemap", "INFO")
        sitemap_url = urljoin(self.base_url, 'sitemap.xml')
        response = self._fetch_url(sitemap_url)
        if not response:
            print_status("Could not find or fetch sitemap.xml", "WARN")
            return

        try:
            content = response.content
            if response.url.endswith('.gz') or 'gzip' in response.headers.get('Content-Type', ''):
                content = gzip.decompress(content)

            root = ET.fromstring(content)
            namespace = root.tag.split('}')[0][1:] if '}' in root.tag else ''

            # Simple parser for urlset, not handling sitemap indexes for this focused check
            for url_node in root.findall(f'.//{{{namespace}}}loc'):
                if len(self.urls_to_check) < self.max_pages_to_check:
                    self.urls_to_check.append(url_node.text.strip())
            print_status(f"Found {len(self.urls_to_check)} URLs in sitemap", "PASS")
        except Exception as e:
            self.report["issues"].append(f"Failed to parse sitemap.xml: {e}")
            print_status("Sitemap parsing failed", "WARN")

    def analyze_page(self, url):
        """Analyzes a single page for its canonical tag."""
        print(f"\n-> Checking: {url}")
        self.report["pages_checked"] += 1

        response = self._fetch_url(url, 'GET')
        if not response or 'text/html' not in response.headers.get('Content-Type', ''):
            print_status("Page is not valid HTML or is unreachable", "FAIL")
            return

        soup = BeautifulSoup(response.text, 'lxml')
        canonical_tag = soup.find('link', {'rel': 'canonical'})

        if not canonical_tag:
            self.report["issues"].append(f"Missing canonical tag on: {url}")
            print_status("Canonical tag presence", "FAIL")
            return

        self.report["pages_with_canonical"] += 1
        print_status("Canonical tag presence", "PASS")

        href = canonical_tag.get('href')
        if not href:
            self.report["issues"].append(f"Canonical tag has empty href on: {url}")
            print_status("Canonical href validity", "FAIL")
            return

        # *** THIS IS THE CORRECTED LINE ***
        parsed_href = urlparse(href)
        if not (parsed_href.scheme and parsed_href.netloc):
            self.report["issues"].append(f"Relative canonical URL '{href}' found on: {url}")
            print_status("Canonical URL is absolute", "FAIL")
            return

        self.report["pages_with_absolute_canonical"] += 1
        print_status("Canonical URL is absolute", "PASS")

        # Check if the canonical target is accessible
        clean_href = urldefrag(href).url # Remove fragments
        head_response = self._fetch_url(clean_href, 'HEAD')
        if head_response and head_response.status_code == 200:
            self.report["pages_with_valid_canonical_target"] += 1
            print_status("Canonical target accessibility", "PASS")
        else:
            status = head_response.status_code if head_response else 'Unreachable'
            self.report["issues"].append(f"Canonical URL '{href}' is not accessible (Status: {status}) on page: {url}")
            print_status("Canonical target accessibility", "FAIL")

    def run_analysis(self):
        """Main execution logic."""
        print_header("ARI Sub-Pillar 1.3: Canonicalization & Source Singularity")

        self.get_urls_from_sitemap()
        if not self.urls_to_check:
            self.urls_to_check.append(self.base_url)

        print_subheader(f"Analyzing up to {self.max_pages_to_check} pages")
        while self.urls_to_check and len(self.checked_urls) < self.max_pages_to_check:
            url = self.urls_to_check.popleft()
            if url in self.checked_urls:
                continue

            self.checked_urls.add(url)
            self.analyze_page(url)
            time.sleep(0.1)

        self._generate_final_report()
        self._print_final_report()

    def _generate_final_report(self):
        """Calculate final score and populate recommendations."""
        if self.report["pages_checked"] == 0:
            self.report["status"] = "Critical Failure"
            self.report["score"] = 0
            self.report["recommendations"].append("Could not fetch or analyze any pages from the target URL.")
            return

        # Calculate percentages
        p_canonical = self.report["pages_with_canonical"] / self.report["pages_checked"]
        p_absolute = self.report["pages_with_absolute_canonical"] / self.report["pages_with_canonical"] if self.report["pages_with_canonical"] > 0 else 1
        p_valid_target = self.report["pages_with_valid_canonical_target"] / self.report["pages_with_canonical"] if self.report["pages_with_canonical"] > 0 else 1

        # Scoring: 50% for presence, 25% for being absolute, 25% for being valid.
        score = (p_canonical * 50) + (p_canonical * p_absolute * 25) + (p_canonical * p_valid_target * 25)
        self.report["score"] = int(score)

        if score >= 90: self.report["status"] = "Excellent"
        elif score >= 70: self.report["status"] = "Good"
        elif score >= 40: self.report["status"] = "Needs Improvement"
        else: self.report["status"] = "Poor"

        # Recommendations
        if p_canonical < 1.0:
            self.report["recommendations"].append("Implement <link rel='canonical'> tags on all indexable pages.")
        if p_absolute < 1.0:
            self.report["recommendations"].append("Ensure all canonical tag href attributes use absolute URLs, not relative paths.")
        if p_valid_target < 1.0:
            self.report["recommendations"].append("Audit canonical URLs that are broken (e.g., 404s) or redirect, and point them to valid, final destinations.")
        if score < 70:
             self.report["recommendations"].append("Audit for duplicate content across different URL structures (e.g., www vs. non-www, http vs. https).")


    def _print_final_report(self):
        """Prints the final formatted report."""
        print_header("Final Assessment Report")
        print_status("Overall Status", self.report['status'])
        print_status("ARI Score (out of 100)", self.report['score'])

        print_subheader("Summary Statistics")
        print(f"Pages Analyzed: {self.report['pages_checked']}")
        if self.report['pages_checked'] > 0:
            p_canonical = (self.report['pages_with_canonical'] / self.report['pages_checked']) * 100
            print(f"Pages with a Canonical Tag: {self.report['pages_with_canonical']}/{self.report['pages_checked']} ({p_canonical:.1f}%)")
        if self.report['pages_with_canonical'] > 0:
            p_absolute = (self.report['pages_with_absolute_canonical'] / self.report['pages_with_canonical']) * 100
            p_valid = (self.report['pages_with_valid_canonical_target'] / self.report['pages_with_canonical']) * 100
            print(f"Absolute Canonical URLs: {self.report['pages_with_absolute_canonical']}/{self.report['pages_with_canonical']} ({p_absolute:.1f}%)")
            print(f"Accessible Canonical Targets: {self.report['pages_with_valid_canonical_target']}/{self.report['pages_with_canonical']} ({p_valid:.1f}%)")

        if self.report["issues"]:
            print_subheader("Detected Issues")
            for issue in self.report["issues"][:5]: # Show top 5 issues
                print(f"  - {issue}")

        if self.report["recommendations"]:
            print_subheader("Recommendations (Based on ARI v10.0)")
            for rec in self.report["recommendations"]:
                print_recommendation(rec)



In [5]:
# --- MAIN PROGRAM ---

print("CanonicalizationAnalyzer" + "="*60)

target_url = input("Enter URL: ")

# Instantiate the analyzer with the given target URL
analyzer = CanonicalizationAnalyzer(base_url=target_url, max_pages_to_check=25)

# Run the full analysis
analyzer.run_analysis()

# Print the raw report dict if needed
print("\n" + "="*60)
print("RAW REPORT DATA (for debugging / API use)")
print("="*60)
print(analyzer.report)


CanonicalizationAnalyzer============================================================


Enter URL:  http://localhost:3000/ari?pillar=geo



 ARI Sub-Pillar 1.3: Canonicalization & Source Singularity
Attempting to fetch URLs from sitemap         [INFO]
Sitemap parsing failed                        [WARN]

------------------------------------------------------------
 Analyzing up to 25 pages
------------------------------------------------------------

-> Checking: http://localhost:3000
Canonical tag presence                        [FAIL]

 Final Assessment Report
Overall Status                                [Poor]
ARI Score (out of 100)                        [0]

------------------------------------------------------------
 Summary Statistics
------------------------------------------------------------
Pages Analyzed: 1
Pages with a Canonical Tag: 0/1 (0.0%)

------------------------------------------------------------
 Detected Issues
------------------------------------------------------------
  - Failed to parse sitemap.xml: syntax error: line 1, column 0
  - Missing canonical tag on: http://localhost:3000

----------

In [ ]:
--------------WHAT IS MISSING IN CURRENT CODE -----------------------
1️⃣ NO CHECK for Canonical SELF-REFERENCE (CRITICAL)

        Every page should canonicalize to itself, unless it is:
        
       A. paginated
        
       B. filtered
        
       C. duplicate variant
        
       D. alternate-language
        
       E. A/B test bucket
        
        Right now,  code never checks this.
        
        WE MUST verify:
        clean_current = urldefrag(response.url).url
        clean_canonical = urldefrag(href).url
        
        if clean_current != clean_canonical:
            flag “non-self-referencing canonical”
        
        
        This is a major signal of duplicate content.

2️⃣ NO CHECK for MULTIPLE CANONICAL TAGS (CRITICAL)

    If a page has:
    
    <link rel="canonical" href="...">
    <link rel="canonical" href="...">
    
    
        Or canonical added via:
        
        HTML head
        
        JS injection
        
        HTTP header
        
        → It breaks trust.
        
        analyzer does not detect this.

3️⃣ NO CHECK for CONFlicting Canonical Logic (IMPORTANT)

    should detect pages where:
    
    Canonical points to URL A
    
    But redirects to URL B
    
    But sitemap lists URL C
    
    This is called canonical drift.
    
    Your code only checks HEAD 200 — not consistency.

4️⃣ NO CHECK for CANONICAL REDIRECTS (IMPORTANT)

    Canonical URL should not redirect.
    
    Right now, you only do:
    
    HEAD → status_code == 200
    
    
    But if canonical returns:
    
    301
    
    302
    
    307
    
    You treat it as FAIL, but you do not differentiate WHY.
    
    You need:
    
    Direct 200 PASS
    
    3xx → WARN (fix target)
    
    4xx/5xx → FAIL

5️⃣ NO CHECK for PARAMETER CANONICALIZATION (IMPORTANT)

    URLs like:
    
    /?ref=twitter
    /?utm_source=x
    /?session=abc
    /?sort=price
    
    Your analyzer does not check whether canonical removes parameters.
    
    You should detect pages where:
    
    URL has params
    
    Canonical should strip them
    
    But doesn’t

6️⃣ NO CHECK: Sitemap vs Canonical Alignment (ADVANCED)

    You never check:
    
    “Does the sitemap URL match the canonical target?”
    
    This is a major governance error.
    
    If sitemap lists:
    
    https://example.com/blog/article
    
    
    But canonical says:
    
    https://example.com/articles/123/
    
    
    → The AI agent doesn't know which is authoritative.
    
    You must detect this mismatch.

7️⃣ NO CHECK for WWW vs NON-WWW Canonical Conflicts (ADVANCED)

    If:
    
    Page lives at www
    
    Canonical points to non-www
    
    Or vice-versa.
    
    You currently don't validate domain alignment.

8️⃣ NO CHECK for HTTP vs HTTPS Canonical Conflicts (ADVANCED)

    Example conflict:
    
    Live page:
    https://example.com/about
    
    Canonical:
    http://example.com/about
    
    This is a trust degradation.

9️⃣ NO CHECK for CANONICAL CYCLES (ADVANCED)

    Case:
    
    Page A canonical → Page B
    
    Page B canonical → Page A
    
    Your analyzer does not detect loops.

🔟 NO PLURAL CANONICAL ENFORCEMENT RULES (ADVANCED)

    You should validate:
    
    Canonical always points to final resolved URL
    
    Canonical never points to:
    – paginated pages
    – filtered pages
    – search results
    – admin URLs
    
    Current code does none of this.

1️⃣1️⃣ NO CONTENT SIMILARITY CHECK (OPTIONAL / AI-HEAVY)

    True duplicate detection requires:
    
    Fetch canonical page
    
    Compare textual similarity
    
    If similarity < 85% → canonical mismatch
    
    This is advanced but expected in ARI v10 Full Suite.



1️⃣2️⃣ NO DETECTION OF META OR HTTP-HEADER CANONICAL (ADVANCED)

    Some sites use:
    
    <meta name="canonical"> (bad)
    
    Link: <url>; rel="canonical" (HTTP header level)
    
    Your analyzer never checks response headers.


Summary — Here Are the Missing Pieces

| Area                  | Missing Check              | Impact   |
| --------------------- | -------------------------- | -------- |
| Canonical correctness | Self-reference             | CRITICAL |
| Structural validation | Multiple canonical tags    | CRITICAL |
| Consistency           | Sitemap ↔ Canonical        | HIGH     |
| URL hygiene           | Redirecting canonicals     | HIGH     |
| URL integrity         | Parameter stripping        | HIGH     |
| Protocol/Domain       | HTTPS/HTTP & WWW alignment | HIGH     |
| Loop detection        | Canonical cycles           | MEDIUM   |
| Header-level signals  | HTTP canonical             | MEDIUM   |
| Content validation    | Real duplicate analysis    | OPTIONAL |


===========TL;DR — What’s Missing ===

    Your analyzer checks presence, absolute, reachable.
    
    It DOES NOT check:
    
    1. correctness
    2. consistency
    3. conflicts
    4. canonical drift
    5. alignment
    6. multi-source signals
    7. URL normalization
    8. real duplication

In [ ]:
----------IMPROVE VERSION ----------------------

In [15]:
import requests
import xml.etree.ElementTree as ET
import gzip
from io import BytesIO
from urllib.parse import urlparse, urljoin, urldefrag, parse_qs
from bs4 import BeautifulSoup
import time
import collections
from typing import Optional, Dict, List, Tuple

# Helper functions for formatted output
def print_header(text):
    print("\n" + "="*70)
    print(f" {text}")
    print("="*70)

def print_subheader(text):
    print("\n" + "-"*70)
    print(f" {text}")
    print("-"*70)

def print_status(message, status):
    padded_message = f"{message:<50}"
    if status in ["FAIL", "CRITICAL"]:
        status_str = f"[\033[91m{status}\033[0m]"
    elif status == "WARN":
        status_str = f"[\033[93m{status}\033[0m]"
    elif status in ["PASS", "INFO"]:
        status_str = f"[\033[92m{status}\033[0m]"
    else:
        status_str = f"[{status}]"
    print(f"{padded_message} {status_str}")

def print_recommendation(rec):
    print(f"  → {rec}")

class EnhancedCanonicalizationAnalyzer:
    """
    Enhanced analyzer for ARI v10.0 Pillar 1, Sub-pillar 3.
    Checks canonicalization correctness, consistency, and conflicts.
    """
    def __init__(self, base_url, max_pages_to_check=25):
        self.base_url = self._format_base_url(base_url)
        self.max_pages_to_check = max_pages_to_check
        self.urls_to_check = collections.deque()
        self.checked_urls = set()
        self.sitemap_urls = set()  # Track URLs from sitemap
        
        # Enhanced tracking
        self.report = {
            "pages_checked": 0,
            "pages_with_canonical": 0,
            "pages_with_absolute_canonical": 0,
            "pages_with_valid_canonical_target": 0,
            "self_referencing_canonicals": 0,
            "multiple_canonical_tags": 0,
            "canonical_redirects": 0,
            "http_header_canonicals": 0,
            "canonical_cycles_detected": 0,
            "www_mismatches": 0,
            "protocol_mismatches": 0,
            "parameter_issues": 0,
            "sitemap_canonical_mismatches": 0,
            "issues": [],
            "critical_issues": [],
            "warnings": [],
            "recommendations": [],
            "score": 0,
            "status": "Not Assessed"
        }
        
        self.session = requests.Session()
        self.session.headers.update({
            'User-Agent': 'ARI-Enhanced-Canonical-Analyzer/2.0',
            'Accept-Language': 'en-US,en;q=0.9',
            'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
        })
        
        # Track canonical relationships for cycle detection
        self.canonical_graph = {}

    def _format_base_url(self, url):
        """Normalize base URL to include scheme."""
        parsed = urlparse(url)
        if not parsed.scheme:
            url = "https://" + url
            parsed = urlparse(url)
        return f"{parsed.scheme}://{parsed.netloc}"

    def _normalize_url(self, url):
        """
        Normalize URL by:
        - Removing fragment (#anchor)
        - Converting to lowercase for domain
        - Removing trailing slash for consistency (except root)
        """
        clean_url = urldefrag(url).url
        parsed = urlparse(clean_url)
        
        # Normalize domain to lowercase
        netloc = parsed.netloc.lower()
        
        # Remove trailing slash except for root path
        path = parsed.path
        if path != '/' and path.endswith('/'):
            path = path.rstrip('/')
        
        normalized = f"{parsed.scheme}://{netloc}{path}"
        if parsed.query:
            normalized += f"?{parsed.query}"
        
        return normalized

    def _fetch_url(self, url, method='GET', allow_redirects=True):
        """Fetch URL with error handling."""
        try:
            if method == 'GET':
                response = self.session.get(url, timeout=10, allow_redirects=allow_redirects)
            elif method == 'HEAD':
                response = self.session.head(url, timeout=10, allow_redirects=allow_redirects)
            return response
        except requests.exceptions.RequestException as e:
            self.report["issues"].append(f"Network error for {url}: {str(e)[:100]}")
            return None

    def _check_http_header_canonical(self, response) -> Optional[str]:
        """Check for canonical URL in HTTP Link header."""
        link_header = response.headers.get('Link', '')
        if 'rel="canonical"' in link_header or "rel='canonical'" in link_header:
            # Parse Link header (simplified parsing)
            parts = link_header.split(',')
            for part in parts:
                if 'canonical' in part:
                    url_part = part.split(';')[0].strip()
                    canonical_url = url_part.strip('<>')
                    return canonical_url
        return None

    def get_urls_from_sitemap(self):
        """Fetch URLs from sitemap.xml."""
        print_status("Fetching URLs from sitemap", "INFO")
        sitemap_url = urljoin(self.base_url, 'sitemap.xml')
        response = self._fetch_url(sitemap_url)
        
        if not response or response.status_code != 200:
            print_status("Could not fetch sitemap.xml", "WARN")
            return

        try:
            content = response.content
            if response.url.endswith('.gz') or 'gzip' in response.headers.get('Content-Type', ''):
                content = gzip.decompress(content)

            root = ET.fromstring(content)
            namespace = root.tag.split('}')[0][1:] if '}' in root.tag else ''

            for url_node in root.findall(f'.//{{{namespace}}}loc'):
                url = url_node.text.strip()
                self.sitemap_urls.add(self._normalize_url(url))
                if len(self.urls_to_check) < self.max_pages_to_check:
                    self.urls_to_check.append(url)
                    
            print_status(f"Found {len(self.sitemap_urls)} URLs in sitemap", "PASS")
        except Exception as e:
            self.report["warnings"].append(f"Sitemap parsing error: {str(e)[:100]}")
            print_status("Sitemap parsing failed", "WARN")

    def _has_tracking_parameters(self, url):
        """Check if URL has tracking/session parameters."""
        parsed = urlparse(url)
        if not parsed.query:
            return False
        
        params = parse_qs(parsed.query)
        tracking_params = ['utm_source', 'utm_medium', 'utm_campaign', 'ref', 
                          'source', 'session', 'sid', 'fbclid', 'gclid']
        
        return any(param in params for param in tracking_params)

    def analyze_page(self, url):
        """Comprehensive analysis of a single page's canonicalization."""
        print(f"\n→ Analyzing: {url}")
        self.report["pages_checked"] += 1
        
        # Fetch the page
        response = self._fetch_url(url, 'GET')
        if not response or 'text/html' not in response.headers.get('Content-Type', ''):
            print_status("Not valid HTML or unreachable", "FAIL")
            return

        clean_current_url = self._normalize_url(response.url)
        
        # Parse HTML
        soup = BeautifulSoup(response.text, 'lxml')
        
        # ═══════════════════════════════════════════════════════════
        # CHECK 1: Multiple Canonical Tags (CRITICAL)
        # ═══════════════════════════════════════════════════════════
        canonical_tags = soup.find_all('link', {'rel': 'canonical'})
        
        if len(canonical_tags) > 1:
            self.report["multiple_canonical_tags"] += 1
            self.report["critical_issues"].append(
                f"CRITICAL: Multiple canonical tags ({len(canonical_tags)}) on {url}"
            )
            print_status("Multiple canonical tags detected", "CRITICAL")
        
        # ═══════════════════════════════════════════════════════════
        # CHECK 2: HTTP Header Canonical (ADVANCED)
        # ═══════════════════════════════════════════════════════════
        http_canonical = self._check_http_header_canonical(response)
        if http_canonical:
            self.report["http_header_canonicals"] += 1
            print_status("HTTP Link header canonical found", "INFO")
        
        # Get primary canonical from HTML
        canonical_tag = canonical_tags[0] if canonical_tags else None
        
        if not canonical_tag:
            self.report["critical_issues"].append(f"Missing canonical tag: {url}")
            print_status("Canonical tag presence", "FAIL")
            return

        self.report["pages_with_canonical"] += 1
        print_status("Canonical tag presence", "PASS")

        href = canonical_tag.get('href', '').strip()
        if not href:
            self.report["critical_issues"].append(f"Empty canonical href on: {url}")
            print_status("Canonical href validity", "FAIL")
            return

        # ═══════════════════════════════════════════════════════════
        # CHECK 3: Absolute URL Check
        # ═══════════════════════════════════════════════════════════
        parsed_href = urlparse(href)
        if not (parsed_href.scheme and parsed_href.netloc):
            self.report["critical_issues"].append(
                f"Relative canonical '{href}' on: {url}"
            )
            print_status("Canonical is absolute", "FAIL")
            return

        self.report["pages_with_absolute_canonical"] += 1
        print_status("Canonical is absolute", "PASS")

        clean_canonical_url = self._normalize_url(href)
        
        # ═══════════════════════════════════════════════════════════
        # CHECK 4: Self-Referencing Canonical (CRITICAL)
        # ═══════════════════════════════════════════════════════════
        if clean_current_url == clean_canonical_url:
            self.report["self_referencing_canonicals"] += 1
            print_status("Self-referencing canonical", "PASS")
        else:
            # Check if this is a legitimate non-self-reference
            # (pagination, filters, variants should point elsewhere)
            is_paginated = any(p in url.lower() for p in ['page=', '/page/', '?p='])
            has_filters = any(f in url.lower() for f in ['filter=', 'sort=', 'category='])
            
            if not (is_paginated or has_filters):
                self.report["warnings"].append(
                    f"Non-self-referencing canonical: {url} → {clean_canonical_url}"
                )
                print_status("Non-self-referencing (check if valid)", "WARN")
            else:
                print_status("Points to canonical version (paginated/filtered)", "INFO")

        # ═══════════════════════════════════════════════════════════
        # CHECK 5: WWW vs Non-WWW Alignment (ADVANCED)
        # ═══════════════════════════════════════════════════════════
        current_domain = urlparse(clean_current_url).netloc
        canonical_domain = urlparse(clean_canonical_url).netloc
        
        if current_domain.replace('www.', '') != canonical_domain.replace('www.', ''):
            if current_domain != canonical_domain:
                self.report["www_mismatches"] += 1
                self.report["warnings"].append(
                    f"Domain mismatch: {current_domain} → {canonical_domain}"
                )
                print_status("WWW/domain consistency", "WARN")

        # ═══════════════════════════════════════════════════════════
        # CHECK 6: HTTP vs HTTPS Protocol (ADVANCED)
        # ═══════════════════════════════════════════════════════════
        current_protocol = urlparse(clean_current_url).scheme
        canonical_protocol = urlparse(clean_canonical_url).scheme
        
        if current_protocol != canonical_protocol:
            self.report["protocol_mismatches"] += 1
            self.report["critical_issues"].append(
                f"Protocol mismatch: {current_protocol} → {canonical_protocol} on {url}"
            )
            print_status("Protocol consistency (HTTP/HTTPS)", "CRITICAL")

        # ═══════════════════════════════════════════════════════════
        # CHECK 7: Parameter Canonicalization (IMPORTANT)
        # ═══════════════════════════════════════════════════════════
        if self._has_tracking_parameters(url):
            canonical_has_params = bool(urlparse(clean_canonical_url).query)
            if canonical_has_params:
                self.report["parameter_issues"] += 1
                self.report["warnings"].append(
                    f"Tracking params not stripped in canonical: {url}"
                )
                print_status("Parameter stripping", "WARN")
            else:
                print_status("Tracking parameters properly removed", "PASS")

        # ═══════════════════════════════════════════════════════════
        # CHECK 8: Canonical Target Accessibility & Redirects (IMPORTANT)
        # ═══════════════════════════════════════════════════════════
        head_response = self._fetch_url(clean_canonical_url, 'HEAD', allow_redirects=False)
        
        if head_response:
            status = head_response.status_code
            
            if status == 200:
                self.report["pages_with_valid_canonical_target"] += 1
                print_status("Canonical target accessible", "PASS")
            elif 300 <= status < 400:
                self.report["canonical_redirects"] += 1
                redirect_location = head_response.headers.get('Location', 'unknown')
                self.report["warnings"].append(
                    f"Canonical redirects ({status}): {clean_canonical_url} → {redirect_location}"
                )
                print_status(f"Canonical redirects ({status})", "WARN")
            elif 400 <= status < 500:
                self.report["critical_issues"].append(
                    f"Canonical returns {status} (client error): {clean_canonical_url}"
                )
                print_status(f"Canonical target broken ({status})", "FAIL")
            else:
                self.report["critical_issues"].append(
                    f"Canonical returns {status} (server error): {clean_canonical_url}"
                )
                print_status(f"Canonical target error ({status})", "FAIL")
        else:
            print_status("Cannot verify canonical target", "WARN")

        # ═══════════════════════════════════════════════════════════
        # CHECK 9: Sitemap vs Canonical Alignment (ADVANCED)
        # ═══════════════════════════════════════════════════════════
        if self.sitemap_urls:
            normalized_sitemap_url = self._normalize_url(url)
            if normalized_sitemap_url in self.sitemap_urls:
                if clean_canonical_url != normalized_sitemap_url:
                    self.report["sitemap_canonical_mismatches"] += 1
                    self.report["warnings"].append(
                        f"Sitemap/canonical mismatch: sitemap has {url}, canonical points to {clean_canonical_url}"
                    )
                    print_status("Sitemap-canonical alignment", "WARN")

        # ═══════════════════════════════════════════════════════════
        # CHECK 10: Track Canonical Graph for Cycle Detection
        # ═══════════════════════════════════════════════════════════
        self.canonical_graph[clean_current_url] = clean_canonical_url

    def _detect_canonical_cycles(self):
        """Detect canonical cycles after all pages analyzed."""
        print_subheader("Detecting Canonical Cycles")
        
        for start_url, canonical_url in self.canonical_graph.items():
            visited = set()
            current = start_url
            
            while current in self.canonical_graph:
                if current in visited:
                    # Cycle detected
                    self.report["canonical_cycles_detected"] += 1
                    cycle_path = " → ".join(list(visited) + [current])
                    self.report["critical_issues"].append(
                        f"CANONICAL CYCLE: {cycle_path}"
                    )
                    print_status(f"Cycle detected starting at {start_url}", "CRITICAL")
                    break
                
                visited.add(current)
                next_url = self.canonical_graph.get(current)
                
                if not next_url or next_url == current:
                    break
                    
                current = next_url

    def run_analysis(self):
        """Main execution logic."""
        print_header("ARI v10.0 Enhanced Canonicalization Analyzer")
        print("Pillar 1, Sub-pillar 3: Canonicalization & Source Singularity\n")
        
        # Fetch sitemap
        self.get_urls_from_sitemap()
        
        # Fallback to homepage if no sitemap
        if not self.urls_to_check:
            self.urls_to_check.append(self.base_url)

        print_subheader(f"Analyzing up to {self.max_pages_to_check} pages")
        
        # Analyze pages
        while self.urls_to_check and len(self.checked_urls) < self.max_pages_to_check:
            url = self.urls_to_check.popleft()
            if url in self.checked_urls:
                continue
            
            self.checked_urls.add(url)
            self.analyze_page(url)
            time.sleep(0.2)  # Rate limiting

        # Post-analysis checks
        self._detect_canonical_cycles()
        self._generate_final_report()
        self._print_final_report()

    def _generate_final_report(self):
        """Calculate final score and generate recommendations."""
        if self.report["pages_checked"] == 0:
            self.report["status"] = "Critical Failure"
            self.report["score"] = 0
            self.report["recommendations"].append(
                "Could not analyze any pages. Check URL accessibility."
            )
            return

        # Scoring weights (total 100 points)
        checked = self.report["pages_checked"]
        
        # Base metrics
        p_canonical = self.report["pages_with_canonical"] / checked
        p_absolute = self.report["pages_with_absolute_canonical"] / checked
        p_valid = self.report["pages_with_valid_canonical_target"] / checked
        p_self_ref = self.report["self_referencing_canonicals"] / checked if checked > 0 else 0
        
        # Penalty metrics
        critical_penalty = min(30, self.report["multiple_canonical_tags"] * 10 + 
                              len(self.report["critical_issues"]) * 2)
        warning_penalty = min(20, len(self.report["warnings"]) * 1)
        
        # Calculate score
        base_score = (p_canonical * 30 + p_absolute * 20 + p_valid * 20 + p_self_ref * 30)
        final_score = max(0, base_score - critical_penalty - warning_penalty)
        
        self.report["score"] = int(final_score)
        
        # Status determination
        if final_score >= 90:
            self.report["status"] = "Excellent"
        elif final_score >= 75:
            self.report["status"] = "Good"
        elif final_score >= 50:
            self.report["status"] = "Needs Improvement"
        else:
            self.report["status"] = "Poor"

        # Generate recommendations
        if p_canonical < 1.0:
            self.report["recommendations"].append(
                "Add <link rel='canonical'> tags to all indexable pages"
            )
        
        if self.report["multiple_canonical_tags"] > 0:
            self.report["recommendations"].append(
                "CRITICAL: Remove duplicate canonical tags - only one per page"
            )
        
        if p_self_ref < 0.8:
            self.report["recommendations"].append(
                "Ensure canonical tags self-reference unless page is paginated/filtered"
            )
        
        if self.report["protocol_mismatches"] > 0:
            self.report["recommendations"].append(
                "Fix HTTP/HTTPS protocol mismatches in canonical URLs"
            )
        
        if self.report["canonical_redirects"] > 0:
            self.report["recommendations"].append(
                "Update canonical URLs to point directly to final destination (avoid redirects)"
            )
        
        if self.report["sitemap_canonical_mismatches"] > 0:
            self.report["recommendations"].append(
                "Align sitemap URLs with canonical targets for consistency"
            )
        
        if self.report["canonical_cycles_detected"] > 0:
            self.report["recommendations"].append(
                "CRITICAL: Break canonical cycles - pages pointing to each other"
            )
        
        if self.report["parameter_issues"] > 0:
            self.report["recommendations"].append(
                "Strip tracking/session parameters in canonical URLs"
            )

    def _print_final_report(self):
        """Print comprehensive final report."""
        print_header("Final Assessment Report")
        
        # Overall status
        print_status("Overall Status", self.report['status'])
        print_status(f"ARI Score (out of 100)", str(self.report['score']))
        
        # Summary statistics
        print_subheader("Analysis Summary")
        print(f"  Pages Analyzed: {self.report['pages_checked']}")
        print(f"  Pages with Canonical: {self.report['pages_with_canonical']}")
        print(f"  Self-Referencing: {self.report['self_referencing_canonicals']}")
        print(f"  Valid Targets: {self.report['pages_with_valid_canonical_target']}")
        
        # Issues summary
        print_subheader("Issues Detected")
        print(f"  Critical Issues: {len(self.report['critical_issues'])}")
        print(f"  Warnings: {len(self.report['warnings'])}")
        print(f"  Multiple Canonical Tags: {self.report['multiple_canonical_tags']}")
        print(f"  Canonical Cycles: {self.report['canonical_cycles_detected']}")
        print(f"  Canonical Redirects: {self.report['canonical_redirects']}")
        print(f"  Protocol Mismatches: {self.report['protocol_mismatches']}")
        print(f"  Sitemap Mismatches: {self.report['sitemap_canonical_mismatches']}")
        
        # Critical issues details
        if self.report["critical_issues"]:
            print_subheader("Critical Issues (Top 5)")
            for issue in self.report["critical_issues"][:5]:
                print(f"  ⚠️  {issue}")
        
        # Warnings details
        if self.report["warnings"]:
            print_subheader("Warnings (Top 5)")
            for warning in self.report["warnings"][:5]:
                print(f"  ⚡ {warning}")
        
        # Recommendations
        if self.report["recommendations"]:
            print_subheader("Actionable Recommendations")
            for i, rec in enumerate(self.report["recommendations"], 1):
                print_recommendation(f"{i}. {rec}")



In [19]:
# Simple header
print("EnhancedCanonicalizationAnalyzer" + "="*60)

# Get URL from user
target_url = input("Enter URL: ").strip()

# Instantiate the analyzer with your URL
analyzer = EnhancedCanonicalizationAnalyzer(base_url=target_url, max_pages_to_check=25)

# Run the analysis
analyzer.run_analysis()

# Print structured raw report for dev / API use
print("\n" + "="*60)
print("RAW REPORT OBJECT")
print("="*60)
print(analyzer.report)

EnhancedCanonicalizationAnalyzer============================================================


Enter URL:   https://vercel.com/docs



 ARI v10.0 Enhanced Canonicalization Analyzer
Pillar 1, Sub-pillar 3: Canonicalization & Source Singularity

Fetching URLs from sitemap                         [INFO]
Found 3803 URLs in sitemap                         [PASS]

----------------------------------------------------------------------
 Analyzing up to 25 pages
----------------------------------------------------------------------

→ Analyzing: https://vercel.com/
Canonical tag presence                             [PASS]
Canonical is absolute                              [PASS]
Non-self-referencing (check if valid)              [WARN]
Canonical target accessible                        [PASS]
Sitemap-canonical alignment                        [WARN]

→ Analyzing: https://vercel.com/about
Canonical tag presence                             [PASS]
Canonical is absolute                              [PASS]
Self-referencing canonical                         [PASS]
Canonical target accessible                        [PASS]

→ Analyzi